<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
    <h1>
    GDP Vintages, Releases and Revisions datasets
    </h1>
</div>

<div style="text-align: center; font-family: 'charter bt pro roman'; color: rgb(0, 65, 75);">
<h3>
Documentation
<br>
____________________
<br>
</h3>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    This 
    <span style="color: rgb(0, 65, 75);">jupyter notebook</span>
    provides a step-by-step guide to <b>data building</b> regarding the project <b>'Revisiones y sesgos en las estimaciones preliminares del PBI en el Perú'</b>. This guide covers the creation of GDP mid-term revision dataset for each sector. A key step is the construction at par of what we will call “The ‘t+h’ structure”. This dataset is similar to that of the GDP growth vintages by sector, but instead of growth rate values, it contains values of type “t+h”, where h indicates how many months have passed since the preliminary growth rate was first published; that is, this jupyter notebook also covers the creation of vintages datasets of growth rates associated with a horizon (<b>h</b>).
</div>

<div style="text-align: center; font-family: 'PT Serif Pro Book'; color: rgb(0, 65, 75); font-size: 16px;">
    Jason Cruz
    <br>
    <a href="mailto:jj.cruza@up.edu.pe" style="color: rgb(0, 153, 123); font-size: 16px;">
        jj.cruza@up.edu.pe
    </a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;line-height: 1.5;">
<span style="font-size: 34px;">&#128452;</span> The 't+h' structure should be available for all sectors and frequencies.
    <br>
    <span style="font-size: 24px;">&#8987;</span> Available since <b>1994-2024</b> (Table 1) and since <b>1997-2024</b> (Table 2). 
    <br>
</div>

<div style="font-family: Amaya; text-align: left; color: rgb(0, 65, 75); font-size:16px">The following <b>outline is functional</b>. By utilising the provided buttons, users are able to enhance their experience by browsing this script.<div/>

<div id="outilne">
   <!-- Contenido de la celda de destino -->
</div>

<div style="background-color: #292929; padding: 10px; line-height: 1.5; font-family: 'PT Serif Pro Book';">
    <h2 style="text-align: left; color: #E0E0E0;">
        Outline
    </h2>
    <br>
    <a href="#libraries" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Libraries</a>
    <br>
    <a href="#setup" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        Initial set-up</a>
    <br>
    <a href="#1" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        1. Economic sector selector</a>
    <br>
    <a href="#2" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        2. Create horizon datasets</a>
    <br>
    <a href="#2.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#2.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        2.2. Creating horizon dataset step by step.</a> 
    <br>
    <a href="#3" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        3. Create base year datasets</a>
    <br>
    <a href="#3.1." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.1. Loading growth rate datasets from postgresql.</a>
    <br>
    <a href="#3.2." style="color: #94FFD8; font-size: 16px; margin-left: 20px;">
        3.2. Creating base year dataset.</a> 
    <br>
    <a href="#4" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        4. Remove observations affected by base year</a>
    <br>
    <a href="#5" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        5. Create growth rates by horizon dataset</a>
    <br>
    <a href="#6" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        6. Create vintages and releases datasets</a>
    <br>
    <a href="#7" style="color: #E0E0E0; font-size: 18px; margin-left: 0px;">
        7. Loading to SQL</a>
    <br>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Any questions or issues regarding the coding, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"><span style="font-size: 24px;">&#x2709;</span>
    </a>.
    <div/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    If you don't have the libraries below, please use the following code (as example) to install the required libraries.
    <div/>

In [ ]:
#!pip install os # Comment this code with "#" if you have already installed this library.

<div id="libraries">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Libraries
    </h2>
    <div/>

In [ ]:
# POSTGRESSQL
import os
from sqlalchemy import create_engine

# HORIZON DATASETS
import pandas as pd
import numpy as np
import re


<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="setup">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark;">
    <h2>
    Initial set-up
    </h2>
    <div/>

<p style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> The following function will establish a connection to the <code>gdp_revisions_datasets</code> database in <code>PostgreSQL</code>. The <b>input data</b> used in this jupyter notebook will be loaded from this <code>PostgreSQL</code> database, and similarly, all <b>output data</b> generated by this jupyter notebook will be stored in that database. Ensure that you set the necessary parameters to access the server once you have obtained the required permissions.<p/>
    
<p style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
To request permissions, please email Jason Cruz <a href="mailto:jj.cruza@alum.up.edu.pe" style="color: rgb(0, 153, 123); text-decoration: none;"> <span style="font-size: 24px;">&#x2709;</span>
    </a>.
<p/>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    <span style="font-size: 24px; color: #FFA823; font-weight: bold;">&#9888;</span>
    Enter your user credentials to acces to SQL.
    <div/>

In [ ]:
def create_sqlalchemy_engine():
    """
    Function to create an SQLAlchemy engine using environment variables.
    
    Returns:
        engine: SQLAlchemy engine object.
    """
    # Get environment variables
    user = os.environ.get('CIUP_SQL_USER')  # Get the SQL user from environment variables
    password = os.environ.get('CIUP_SQL_PASS')  # Get the SQL password from environment variables
    host = os.environ.get('CIUP_SQL_HOST')  # Get the SQL host from environment variables
    port = 5432  # Set the SQL port to 5432
    database = 'gdp_revisions_datasets'  # Set the database name 'gdp_revisions_datasets' from SQL

    # Check if all environment variables are defined
    if not all([host, user, password]):
        raise ValueError("Some environment variables are missing (CIUP_SQL_HOST, CIUP_SQL_USER, CIUP_SQL_PASS)")

    # Create connection string
    connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"

    # Create SQLAlchemy engine
    engine = create_engine(connection_string)
    
    return engine

<div style="text-align: left;">
    <span style="font-size: 24px; color: rgb(255, 32, 78); font-weight: bold;">&#9888;</span>
    <span style="font-family: PT Serif Pro Book; color: black; font-size: 16px;">
        Import all other functions required by this jupyter notebook.
    </span>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color:dark; font-size:16px"> Please, check the script <code>gdp_revisions_datasets_functions.py</code> which contains all the functions required by this jupyter notebook. The functions there are ordered according to the <a href="#outilne" style="color: #3d30a2;">sections</a> of this jupyter notebok.<div/>

In [ ]:
from gdp_revisions_datasets_functions import *

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="1">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">1.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Economic sector and data frequency selector</span></h1>

<div id="steps-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-1-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Select economic sector</a>
    <br>
    <a href="#step-1-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Select frequency</a>
</div>

<div id="step-1-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Select economic sector</span>
  </div>

In [ ]:
# Call the function to show the popup window
sector = show_option_window()
print("Selected economic sector:", sector)

<div id="step-1-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Select frequency</span>
  </div>

In [ ]:
# Call the function to show the popup window
frequency = show_frequency_window()
print("Selected frequency:", frequency)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-1" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-1" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book; color: dark;">2.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create horizon datasets</span></h1>

<div id="2.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [ ]:
# Connect to SQL
engine = create_sqlalchemy_engine()

# SQL Query
query = f"SELECT * FROM {sector}_{frequency}_growth_rates;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Comment the code below if you want the default option (display rows and columns of the dataframe in a limited way)
    <div/>

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [ ]:
# Read growth rates dataset as DataFrame
globals()[f'{sector}_{frequency}_growth_rates'] = pd.read_sql(query, engine)
growth_rates = globals()[f'{sector}_{frequency}_growth_rates']
growth_rates_df = growth_rates.copy()
growth_rates_df.head(10)

In [ ]:
# Find duplicates of id_ns within the same year
duplicated_rows = growth_rates_df[growth_rates_df.duplicated(subset=['year', 'id_ns'], keep=False)]
duplicated_rows

In [ ]:
growth_rates_df.iloc[:10,:5]

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="2.2.">
   <!-- Contenido de la celda de destino -->
</div>

<div id="2.3.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">2.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating horizon dataset step by step
    </span>
    </h2>

<div id="steps-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#step-2-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Replace decimal values by “t+h” values only in the rows representing a new rung</a>
    <br>
    <a href="#step-2-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Concatenate first 3 columns: year, date, id_ns </a>
    <br>
    <a href="#step-2-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Convert columns to string type</a>
    <br>
    <a href="#step-2-4" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values </a>
    <br>
    <a href="#step-2-5" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file </a>
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
   <span>Set key variables to fill the growth_rates with 't+h' values</span>
  </div>

In [ ]:
# Call the function to show the h_initial window
h_initial = show_h_initial_window()
print("Selected h_initial:", h_initial)

In [ ]:
# Call the function to show the start_row window
start_row = show_start_row_window()
print("Selected start_row:", start_row)

In [ ]:
# Define the mapping of frequencies to h_counter values
frequency_mapping = {
    'monthly': 1,
    'quarterly': 3,
    'annual': 12
}

# Get the appropriate h_counter value based on the selected frequency
h_counter = frequency_mapping.get(frequency)
print("Selected h_counter:", h_counter)

<div id="step-2-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Replace decimal values by “t+h” values only in the rows representing a new rung</span>
  </div>

In [ ]:
horizon = replace_horizon(growth_rates_df.iloc[:, 3:], start_row, h_initial, h_counter)
horizon_df = horizon.copy()
horizon_df.head(10)

<div id="step-2-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Concatenate first 3 columns: year, date, id_ns</span>
  </div>

In [ ]:
# Get the first three columns of the original DataFrame
first_3_columns = growth_rates_df.iloc[:, :3]

# Concatenate the first three columns with h_{sector}_{frequency}_growth_rates
horizon_df = pd.concat([first_3_columns, horizon_df], axis=1)
horizon_df.head(20)

<div id="step-2-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Convert columns to string type</span>
  </div>

In [ ]:
horizon_df = columns_str(horizon_df)
horizon_df.head(20)

<div id="step-2-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10125;</span> Spreads the "t+h" values over the remaining decimal values
  </div>

In [ ]:
horizon_df = replace_horizon_1(horizon_df)
horizon_df.head(20)

<div id="step-2-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10126;</span> Exporting to excel file
  </div>

In [ ]:
#def export_to_excel(df, filename):
#    # Exportar el DataFrame como un archivo Excel
#    df.to_excel(filename, index=False)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-2" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-2" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="3">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create base year datasets</span></h1>

<div id="3.1.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.1. </span> <span style = "color: dark; font-family: PT Serif Pro Book;">Loading growth rate datasets from <code>PostgresSQL</code></span></h2>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Connect to SQL.
    <div/>

In [ ]:
# SQL Query
query = f"SELECT * FROM ns_base_year;" # Please change your query to PosgtresSQL as you see fit

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Check the loaded dataframe.
    <div/>

In [ ]:
# Read growth rates dataset as DataFrame
ns_base_year = pd.read_sql(query, engine)
ns_base_year_df = ns_base_year.copy()
ns_base_year_df.head(10)

<div id="3.2.">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">3.2.</span>
    <span style = "color: dark; font-family: PT Serif Pro Book;">
    Creating base year dataset
    </span>
    </h2>

In [ ]:
base_year_df = replace_floats_with_base_year(ns_base_year_df, growth_rates_df)

In [ ]:
base_year_df.head(10)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [ ]:
# Export to excel file
#with pd.ExcelWriter('base_year_df.xlsx') as writer:
#    base_year_df.to_excel(writer, sheet_name='base_year_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="4">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">4.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Remove observations affected by base year</span></h1>

<div id="steps-4">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary to match observations affected by base year</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Remove observations affected by base year</a>
    <br>
</div>

<div id="st-4-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary to match observations affected by base year</span>
  </div>

In [ ]:
base_year_dictionary = create_dic_base_year(base_year_df)

In [ ]:
base_year_dictionary

<div id="st-4-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Remove observations affected by base year</span>
  </div>

In [ ]:
input_growth_rates_df = remove_base_year_affected_obs(base_year_dictionary, growth_rates_df)

In [ ]:
input_growth_rates_df.head(30)

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Consider the code below as provisional. Actually, no data should be exported to the current directory folder, all data should be uploaded to SQL.
    <div/>

In [ ]:
# Export to excel file
#with pd.ExcelWriter('input_growth_rates_df.xlsx') as writer:
#    input_growth_rates_df.to_excel(writer, sheet_name='input_growth_rates_df', index=False) # this is an optional, to view data generated

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="5">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">5.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create growth rates by horizon dataset</span></h1>

<div id="steps-5">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-3-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Generating a dictionary with the row indices and their t+h values</a>
    <br>
    <a href="#st-3-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Colapse growth rates by horizon ('t+h') </a>
    <br>
</div>

<div id="st-5-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Generating a dictionary with the row indices and their t+h values</span>
  </div>

In [ ]:
horizon_dictionary = get_last_index_h(horizon_df)

<div id="st-5-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Colapse growth rates by horizon ('t+h')  </span>
  </div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; color: dark; font-size:16px">
    Note that <code>'t'</code> is the date of the first announcement, while <code>'t+h'</code> is 'h' months after the first announcement.
    <div/>

In [ ]:
filtered_h_df = filter_df_by_indices(input_growth_rates_df, horizon_dictionary)
filtered_h_df

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-5" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-5" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="6">
   <!-- Contenido de la celda de destino -->
</div>

<h1><span style = "color: rgb(0, 65, 75); font-family: PT Serif Pro Book;">6.</span> <span style = "color: dark; font-family: PT Serif Pro Book;">Create vintages and releases datasets</span></h1>

<div id="steps-6">
   <!-- Contenido de la celda de destino -->
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <a href="#st-4-1" style="text-decoration: none; color: #006769"> <span style="font-size: 24px; color: rgb(0, 65, 75);">&#10122;</span> Create vintages</a>
    <br>
    <a href="#st-4-2" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> Clean-up vintages dataframe </a>
    <br>
    <a href="#st-4-3" style="text-decoration: none; color: #006769"><span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> Create releases </a>
    <br>
</div>

<div id="st-6-1">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10122;</span> <span>Create vintages</span>
  </div>

In [ ]:
vintages = create_vintages(input_growth_rates_df)
vintages.head(30)

<div id="st-6-2">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10123;</span> <span>Clean-up vintages dataframe</span>
  </div>

In [ ]:
# Call the appropriate function based on frequency and sector
if frequency == 'monthly':
    vintages_df = process_monthly(vintages)
elif frequency == 'quarterly':
    vintages_df = process_quarterly(vintages)
elif frequency == 'annual':
    vintages_df = process_annual(vintages)

In [ ]:
vintages_df.head(30)

<div id="st-6-3">
   <!-- Contenido de la celda de destino -->
</div>

<div style="text-align: left; font-family: 'PT Serif Pro Book'; font-size:22px">
    <span style="font-size: 24px; color: rgb(0, 65, 75)">&#10124;</span> <span>Create releases</span>
  </div>

In [ ]:
releases_df = create_releases(vintages_df, sector)
releases_df.head(30)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#steps-6" style="color: #006769; text-decoration: none;">⮝</a>
    </span> 
    <a href="#steps-6" style="color: #006769; text-decoration: none;">Back to steps.</a>
</div>

#### Check column type

In [ ]:
print(releases_df[f'{sector}_release_1'].dtype)

In [ ]:
print(releases_df['vintages_date'].dtype)

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>

<div id="7">
   <!-- Contenido de la celda de destino -->
</div>

<h2><span style = "color: rgb(0, 65, 75); font-family: charter;">7.</span>
    <span style = "color: dark; font-family: charter;">
    Loading to SQL
    </span>
    </h2>

In [ ]:
#horizon_df.to_sql(f'{sector}_{frequency}_growth_rates_horizon', engine, index=False, if_exists='replace')
#base_year_df.to_sql(f'{sector}_{frequency}_growth_rates_base_year', engine, index=False, if_exists='replace')
filtered_h_df.to_sql(f'{sector}_{frequency}_h', engine, index=False, if_exists='replace')
vintages_df.to_sql(f'{sector}_{frequency}_vintages', engine, index=False, if_exists='replace')
releases_df.to_sql(f'{sector}_{frequency}_releases', engine, index=False, if_exists='replace')

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 20px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">⮝</a>
    </span> 
    <a href="#step-1-1" style="color: rgb(255, 32, 78); text-decoration: none;">Back to <b>select economic sector</b>.</a>
</div>

<div style="font-family: PT Serif Pro Book; text-align: left; color: dark; font-size: 16px;">
    <span style="font-size: 30px; color: rgb(255, 32, 78); font-weight: bold;">
        <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">&#11180;</a>
    </span> 
    <a href="#outilne" style="color: rgb(0, 153, 123); text-decoration: none;">Back to the outline.</a>
</div>